In [ ]:
!pip install tiktoken --quiet
!pip install datasets --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
print("hi")

## **File: previous_chapters**

### **1. tokenizer initialization**

```
# original code
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")
```

```
# modified code
from transformers import PreTrainedTokenizerFast

# Load the tokenizer
tokenizer = PreTrainedTokenizerFast.from_pretrained("Aananda-giri/NepaliBPE")
```

In [ ]:
# sebastian original tokenizer
import tiktoken

txt = "hello world!"

tokenizer = tiktoken.get_encoding("gpt2")

# Encode
token_ids = tokenizer.encode(txt, allowed_special={'<|endoftext|>'})
print(f'Encoded: {type(token_ids)} {token_ids}')

# tokenize
print([tokenizer.decode([token]) for token in tokenizer.encode(txt)])

# Decode
print(f'Decoded: {type(tokenizer.decode(token_ids))} {tokenizer.decode(token_ids)}')


Encoded: <class 'list'> [31373, 995, 0]
['hello', ' world', '!']
Decoded: <class 'str'> hello world!


### **2. tokenizer.encode**

```
# original code
token_ids = tokenizer.encode(txt, allowed_special={'<|endoftext|>'})
```

```
# modified code
token_ids = tokenizer.encode(txt)
```

In [ ]:
# our tokenizer
from transformers import PreTrainedTokenizerFast

# Load the tokenizer
tokenizer = PreTrainedTokenizerFast.from_pretrained("Aananda-giri/NepaliBPE")

text = "तपाईंलाई कस्तो छ?"
token_ids = tokenizer.encode(text)
print(f'Encoded: {type(token_ids)} {token_ids}')

# Tokenize
print(f'tokenized: {tokenizer.tokenize(text)}')

# Decode
print(f'Decoded: {type(tokenizer.decode(token_ids))} {tokenizer.decode(token_ids)}')

Encoded: <class 'list'> [8987, 3199, 186, 1]
tokenized: ['तपाईंलाई</w>', 'कस्तो</w>', 'छ</w>', '<|unk|>']
Decoded: <class 'str'> तपाईंलाई कस्तो छ <|unk|>


### **3. tokenizer.decode**
* leave it as it is


### **4. use pre-tokenized and pre_processed dataset for training**
* modify GPTDatasetV1, create_dataloader_v1 with following code


In [4]:
!pip install datasets --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from torch.utils.data import DataLoader
from datasets import load_dataset
import torch


def create_dataloader_v2(batch_size=4, shuffle=True, drop_last=True, num_workers=0):
    '''
    modified.
    * dont need text data as input
    * dont need max_length and stride as input : they were set during preparing tokenized_datasets
    '''
    # Data loading script from solution3
    base_url = "https://huggingface.co/datasets/Aananda-giri/nepali_llm_datasets/resolve/main/pre_tokenized/"
    data_files = {"train": base_url + "nepberta_train.parquet", "test": base_url + "nepberta_test.parquet"}
    dataset = load_dataset("parquet", data_files=data_files)
    print(f'loaded dataset: {dataset}')


    # Define the custom collate_fn function
    def collate_fn(batch):
        # Extract the 'input_ids' and 'target_ids' from the batch and return them as a list of tensors
        input_ids = [item['input_ids'] for item in batch]
        target_ids = [item['target_ids'] for item in batch]

        # Convert to tensors (if not already)
        input_ids_tensor = torch.stack(input_ids)
        target_ids_tensor = torch.stack(target_ids)

        return [input_ids_tensor, target_ids_tensor]

    # Convert Hugging Face Dataset to PyTorch tensors (we can directly use the dataset as it is already in the correct format)
    dataset.set_format(type="torch", columns=["input_ids", "target_ids"])  # Directly set columns to torch tensors

    # Creating the DataLoader for the 'train' split of the dataset with the custom collate_fn
    train_loader = DataLoader(
        dataset["train"],
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers,
        collate_fn=collate_fn
    )

    test_loader =  DataLoader(
        dataset["test"],
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers,
        collate_fn=collate_fn
    )

    return train_loader, test_loader

train_loader, test_loader = create_dataloader_v2(
    batch_size=4,
    shuffle=True,
    drop_last=True,
    num_workers=0
)
train_loader

nepberta_train.parquet:   0%|          | 0.00/2.93G [00:00<?, ?B/s]

nepberta_test.parquet:   0%|          | 0.00/325M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/23 [00:00<?, ?it/s]

loaded dataset: DatasetDict({
    train: Dataset({
        features: ['input_ids', 'target_ids'],
        num_rows: 700714
    })
    test: Dataset({
        features: ['input_ids', 'target_ids'],
        num_rows: 77858
    })
})


In [4]:
print(len(train_loader))

for input_batch, target_batch in train_loader:
    print(f"len(input_batch):{len(input_batch)}")
    print(f"len(input_batch[0]):{len(input_batch[0])}")
    print(f"input_batch.numel(): {input_batch.numel()}")
    print(f"type(input_batch): {type(input_batch)}", end='\n\n')

    print(f"len(target_batch): {len(target_batch)}")
    print(f"len(input_batch[0]):{len(target_batch[0])}")
    print(f"target_batch.numel(): {target_batch.numel()}")
    print(f"type(target_batch): {type(target_batch)}")
    break


175178
len(input_batch):4
len(input_batch[0]):1024
input_batch.numel(): 4096
type(input_batch): <class 'torch.Tensor'>

len(target_batch): 4
len(input_batch[0]):1024
target_batch.numel(): 4096
type(target_batch): <class 'torch.Tensor'>


### comparision with original dataloader by sebastian

In [6]:
from torch.utils.data import Dataset, DataLoader
from transformers import PreTrainedTokenizerFast
from datasets import load_dataset
import torch

# use streaming=True to avoid downloading entire dataset
nepberta_train = load_dataset("Aananda-giri/nepali_llm_datasets", name="nepberta", streaming=True)['train']
text_data = next(iter(nepberta_train))['text']
print(f"text_chunk length: {len(text_data)}")

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # modified.
        # token_ids = tokenizer.encode(txt, allowed_special={'<|endoftext|>'})
        token_ids = tokenizer.encode(txt)

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]


def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True, num_workers=0):
    # -------------------------------
    # Modofy tokenizer
    # -------------------------------
    # modified. tokenizer initialization
    # tokenizer = tiktoken.get_encoding("gpt2")
    # Load the tokenizer
    tokenizer = PreTrainedTokenizerFast.from_pretrained("Aananda-giri/NepaliBPE")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)

    return dataloader

train_ratio = 0.9
split_idx = int(train_ratio * len(text_data))
train_loader = create_dataloader_v1(
        text_data[:split_idx],
        batch_size=4,
        max_length=1024,
        stride=1024,
        drop_last=True,
        shuffle=True,
        num_workers=0
    )
train_loader

Resolving data files:   0%|          | 0/184 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/46 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/184 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/46 [00:00<?, ?it/s]

text_chunk length: 19626198


In [7]:
print(len(train_loader))

for input_batch, target_batch in train_loader:
    print(f"len(input_batch):{len(input_batch)}")
    print(f"len(input_batch[0]):{len(input_batch[0])}")
    print(f"input_batch.numel(): {input_batch.numel()}")
    print(f"type(input_batch): {type(input_batch)}", end='\n\n')

    print(f"len(target_batch): {len(target_batch)}")
    print(f"len(input_batch[0]):{len(target_batch[0])}")
    print(f"target_batch.numel(): {target_batch.numel()}")
    print(f"type(target_batch): {type(target_batch)}")
    break


771
len(input_batch):4
len(input_batch[0]):1024
input_batch.numel(): 4096
type(input_batch): <class 'torch.Tensor'>

len(target_batch): 4
len(input_batch[0]):1024
target_batch.numel(): 4096
type(target_batch): <class 'torch.Tensor'>


*  using pre-tokenized, we got the similar result as given by sebastian original dataloader

# also replace `create_dataloaders` function from  pretraining_bells_n_whistles file

In [ ]:
def create_dataloaders(train_ratio, batch_size, num_workers=0):
    '''
    we dont need
    text_data
    max_length, stride

    '''
    train_loader, val_loader = create_dataloader_v2(
        batch_size=batch_size,
        shuffle=True,
        drop_last=True,
        num_workers=num_workers,
        train_ratio=train_ratio
    )
    return train_loader, val_loader

## **File: `prepare_dataset.py`**
use nepberta (nepali) dataset instead of project gutengerg

**original code:**

* removes non english text
* combines files from project gutenberg and generate 500Mb chunks with file_name: \<target_dir\>/combined_{file_counter}.txt

* where `file_counter` starts from 1
* and `target_dir = gutenberg_preprocessed/`


**modified code**
* removes non devanagari text
* combines text rows from nepberta and generate 500Mb chunks with file_name \<target_dir\>/chunk_{file_counter}.txt

* **upload to huggingface** for ease of use on different notebooks
* access these chunks from here: https://huggingface.co/datasets/Aananda-giri/nepberta-sample


```
# download 1 chunk of size 500Mb from huggingface
from datasets import load_dataset

num_chunks_to_save = 1
target_dir = 'nepberta_sample'

# Load the dataset from the Hugging Face Hub
sampled_dataset_stream = load_dataset("Aananda-giri/nepberta-sample", split="train", streaming=True)

import os
if not os.path.exists(target_dir):
  os.mkdir(target_dir)

# Save each chunk to a separate text file
for i in range(num_chunks_to_save):
    chunk = next(iter(sampled_dataset_stream))  # Get the next chunk
    with open(os.path.join(target_dir, f"combined_{i+1}.txt"), "w", encoding="utf-8") as file:
        file.write(chunk['text'])
    print(f"Saved chunk {i+1} to chunk_{i+1}.txt")
```

## **File: `pretraining_simple.py`**


**1. tokenizer initialization**

```
# original code
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")
```

```
# modified code
from transformers import PreTrainedTokenizerFast

# Load the tokenizer
tokenizer = PreTrainedTokenizerFast.from_pretrained("Aananda-giri/NepaliBPE")
```

**2. Vocab size:**

```
# original code
GPT_CONFIG_124M = {
            "vocab_size": 50257
            ...
```

```
# modified code
GPT_CONFIG_124M = {
            "vocab_size": 50000
            ...
```

# **Run the code**

In [ ]:
!pip install datasets --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# this location contains files like: prepare_dataset.py, pretraining_simple.py and previous_chapters.py
%cd /content/drive/MyDrive/Research/llm.np/sebastian_gutenberg/

[Errno 2] No such file or directory: '/content/drive/MyDrive/Research/llm.np/sebastian_gutenberg/'
/content


In [ ]:
#  Download dataset
!python3 prepare_dataset.py

README.md: 100% 2.39k/2.39k [00:00<00:00, 13.5MB/s]
Repo card metadata block was not found. Setting CardData to empty.
Saved chunk 1 to chunk_1.txt


In [ ]:
# source: https://github.com/rasbt/LLMs-from-scratch/blob/main/appendix-D/01_main-chapter-code/appendix-D.ipynb
!python pretraining_bells_n_whistles.py \
  --data_dir "nepberta_sample" \
  --n_epochs 5 \
  --batch_size 4 \
  --output_dir model_checkpoints

Total files: 1
Tokenizing file 1 of 1: nepberta_sample/combined_1.txt
1793
Training ...
Ep 1 (Iter 000000): Train loss 10.966, Val loss 10.977
Ep 1 (Iter 000100): Train loss 8.701, Val loss 8.410
Ep 1 (Iter 000200): Train loss 8.213, Val loss 8.032
Ep 1 (Iter 000300): Train loss 8.104, Val loss 7.724
Ep 1 (Iter 000400): Train loss 8.121, Val loss 7.451
Ep 1 (Iter 000500): Train loss 8.122, Val loss 7.085
Ep 1 (Iter 000600): Train loss 7.244, Val loss 6.794
Ep 1 (Iter 000700): Train loss 7.670, Val loss 6.366
Ep 1 (Iter 000800): Train loss 6.663, Val loss 6.292
Ep 1 (Iter 000900): Train loss 7.567, Val loss 6.315
Ep 1 (Iter 001000): Train loss 7.092, Val loss 5.973
Ep 1 (Iter 001100): Train loss 7.603, Val loss 5.891
Ep 1 (Iter 001200): Train loss 6.382, Val loss 5.798
Ep 1 (Iter 001300): Train loss 6.817, Val loss 5.496
Ep 1 (Iter 001400): Train loss 7.190, Val loss 5.462
Ep 1 (Iter 001500): Train loss 6.995, Val loss 5.476
Ep 1 (Iter 001600): Train loss 6.431, Val loss 5.573
Ep 1 (Ite

In [ ]:
# Alternatively: to run the simpler version of code (source: https://github.com/rasbt/LLMs-from-scratch/tree/main/ch05/03_bonus_pretraining_on_gutenberg)
!python pretraining_simple.py \
  --data_dir "nepberta_sample" \
  --n_epochs 5 \
  --batch_size 4 \
  --output_dir model_checkpoints

Total files: 1
Tokenizing file 1 of 1: nepberta_sample/combined_1.txt
Training ...
Ep 1 (Step 0): Train loss 10.700, Val loss 10.543
2024-11-06 15:07:19.711472: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 15:07:19.988826: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 15:07:20.070071: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 15:07:20.482996: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AV

In [ ]:
# !rm -rf model_checkpoints